In [138]:
#libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
import nltk
from sklearn.feature_extraction.text import CountVectorizer

In [139]:
df_train = pd.read_csv("train_with_label.csv")
df_train.head()
print(df_train.head)
print(df_train.shape)
train_labels = df_train['gold_label']
print(train_labels)

<bound method NDFrame.head of         instance_id                                         sentence_1  \
0        train_id_0  Is it in the food supply ? " says David Ropeik...   
1        train_id_1  Hundreds of soldiers were involved , an appare...   
2        train_id_2  And Sen. Michael Crapo , R-Idaho , chairman of...   
3        train_id_3  The gunman , 26-year-old Harold Kilpatrick jnr...   
4        train_id_4  The League of United Latin American Citizens ,...   
...             ...                                                ...   
7796  train_id_7796  Toll last week offered to buy the company for ...   
7797  train_id_7797  Experts say they think better treatment , incl...   
7798  train_id_7798  The Dow Jones industrial average < .DJI > rose...   
7799  train_id_7799  In Sweden , 99 percent of women are literate ,...   
7800  train_id_7800  Tornadoes , up to a foot of rain and hail as b...   

                                             sentence_2  gold_label  
0     The p

In [140]:
universe = pd.read_csv("vocabulary.csv")
print(universe.head)

<bound method NDFrame.head of                                          sentence_number
0      He said he did not think that the Shenzhou V l...
1      Under NASD regulations , Mr. Young can file a ...
2      In Europe , France 's CAC-40 rose 0.6 percent ...
3      Schroeder cancelled his Italian holiday after ...
4      U.S. District Judge William Barbour said he im...
...                                                  ...
31597  " As a professional , " he added , " I think I...
31598  Shares of Mandalay closed down 8 cents at $ 29...
31599  Local police authorities are treating the expl...
31600  There are 625 U.N. troops in Bunia , while the...
31601  . directions both in kbps 200 exceeding speeds...

[31602 rows x 1 columns]>


In [141]:
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

def clean_text(txt):
    text = "".join([c for c in txt if c not in string.punctuation])
    tokens = re.split('\W+', txt)
    txt = " ".join([ps.stem(word) for word in tokens if word not in stopwords])
    txt = txt.lower()
    return txt

In [142]:
df_train['sent1_clean'] = df_train['sentence_1'].apply(lambda x: clean_text(x))
df_train['sent2_clean'] = df_train['sentence_2'].apply(lambda x: clean_text(x))
universe['vocab_clean'] = universe['sentence_number'].apply(lambda x: clean_text(x))
print(universe.head)

<bound method NDFrame.head of                                          sentence_number  \
0      He said he did not think that the Shenzhou V l...   
1      Under NASD regulations , Mr. Young can file a ...   
2      In Europe , France 's CAC-40 rose 0.6 percent ...   
3      Schroeder cancelled his Italian holiday after ...   
4      U.S. District Judge William Barbour said he im...   
...                                                  ...   
31597  " As a professional , " he added , " I think I...   
31598  Shares of Mandalay closed down 8 cents at $ 29...   
31599  Local police authorities are treating the expl...   
31600  There are 625 U.N. troops in Bunia , while the...   
31601  . directions both in kbps 200 exceeding speeds...   

                                             vocab_clean  
0      he said think shenzhou v launch militari appli...  
1      under nasd regul mr young file respons request...  
2      in europ franc cac 40 rose 0 6 percent britain...  
3      schroe

In [143]:
cv = CountVectorizer().fit(universe['vocab_clean']) #learn all tokens in raw document
tokens = cv.get_feature_names()
len(tokens) #this is a list of all unique words


#all the words in all sentences from train, dev, test
#dictionary.values

11354

In [144]:
vectorizer = CountVectorizer(vocabulary = tokens)
X = vectorizer.fit_transform(df_train['sent1_clean'])
traindf1_sent1 = pd.DataFrame(X.toarray(),
                            columns = vectorizer.get_feature_names())
print(traindf1_sent1.shape)

Y = vectorizer.fit_transform(df_train['sent2_clean'])
traindf1_sent2 = pd.DataFrame(X.toarray(),
                            columns = vectorizer.get_feature_names())
print(traindf1_sent2.shape)

#acheived two sparse dfs with same size of universe of words

(7801, 11354)
(7801, 11354)


In [145]:
import numpy as geek

X1 = traindf1_sent1.to_numpy()
X2 = traindf1_sent2.to_numpy()

train_final = geek.add(X1, X2)
print(train_final.max())

#add data frames together to get overlapping words matrix

10


In [146]:
from sklearn.neural_network import MLPClassifier

In [187]:
mlp = MLPClassifier(hidden_layer_sizes=(100,50,3), max_iter = 500, activation = 'relu', solver = 'adam', alpha = 0.001, random_state = 1)
mlp.fit(train_final,train_labels)

MLPClassifier(alpha=0.001, hidden_layer_sizes=(100, 50, 3), max_iter=500,
              random_state=1)

In [149]:
#move into validation set

df_dev = pd.read_csv("dev_with_label.csv")
print(df_dev.head)
print(df_dev.shape)
dev_labels = df_dev['gold_label']
print(dev_labels.shape)

#loaded dev data and defined labels

<bound method NDFrame.head of       instance_id                                         sentence_1  \
0        dev_id_0  He said he did not think that the Shenzhou V l...   
1        dev_id_1  Under NASD regulations , Mr. Young can file a ...   
2        dev_id_2  In Europe , France 's CAC-40 rose 0.6 percent ...   
3        dev_id_3  Schroeder cancelled his Italian holiday after ...   
4        dev_id_4  U.S. District Judge William Barbour said he im...   
...           ...                                                ...   
3995  dev_id_3995  Authorities said the bodies of a man and woman...   
3996  dev_id_3996  The most common side effects after getting the...   
3997  dev_id_3997  Russian cosmonaut Malenchenko achieved a first...   
3998  dev_id_3998  Maddox , who had battled cancer since 1983 , c...   
3999  dev_id_3999  She appeared in federal court there Monday and...   

                                             sentence_2  gold_label  
0     He said he did not think that

In [150]:
#clean sentences

df_dev['sent1_clean'] = df_dev['sentence_1'].apply(lambda x: clean_text(x))
df_dev['sent2_clean'] = df_dev['sentence_2'].apply(lambda x: clean_text(x))

In [151]:
vectorizer_dev = CountVectorizer(vocabulary = tokens)
dev1 = vectorizer_dev.fit_transform(df_dev['sent1_clean'])
devdf1_sent1 = pd.DataFrame(dev1.toarray(),
                            columns = vectorizer.get_feature_names())
print(devdf1_sent1.shape)

dev2 = vectorizer_dev.fit_transform(df_dev['sent2_clean'])
devdf1_sent2 = pd.DataFrame(dev2.toarray(),
                            columns = vectorizer.get_feature_names())
print(devdf1_sent2.shape)


(4000, 11354)
(4000, 11354)


In [153]:
import numpy as geek

D1 = devdf1_sent1.to_numpy()
D2 = devdf1_sent2.to_numpy()

dev_df_final = geek.add(D1, D2)
print(dev_df_final.max())

#add data frames together to get overlapping words matrix

7


In [188]:
prediction = mlp.predict(dev_df_final)
print(prediction.shape)
print(dev_labels.shape)

(4000,)
(4000,)


In [189]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(dev_labels, prediction))

              precision    recall  f1-score   support

           0       0.76      0.91      0.83      3000
           1       0.35      0.15      0.21      1000

    accuracy                           0.72      4000
   macro avg       0.56      0.53      0.52      4000
weighted avg       0.66      0.72      0.67      4000



In [162]:
df_test = pd.read_csv("test_without_label.csv")
print(df_test.head)
print(df_test.shape)

#loaded test data

<bound method NDFrame.head of        instance_id                                         sentence_1  \
0        test_id_0  Jews , Americans and their allies had " evil "...   
1        test_id_1  Zuccarini was ordered held without bail Wednes...   
2        test_id_2  Contribute 2 costs 69 for an individual licens...   
3        test_id_3  The gunman , 26-year-old Harold Kilpatrick jnr...   
4        test_id_4  " The bank requires growth from elsewhere in t...   
...            ...                                                ...   
3995  test_id_3995  " As a professional , " he added , " I think I...   
3996  test_id_3996  Carter , who received the Nobel Peace Prize la...   
3997  test_id_3997  Thompson ordered Moore to remove the monument ...   
3998  test_id_3998  Chief Justice William H. Rehnquist and Justice...   
3999  test_id_3999  A total of 16.3 million lines provided advance...   

                                             sentence_2  
0     . said Amrozi , Indonesia lik

In [163]:
#clean sentences

df_test['sent1_clean'] = df_test['sentence_1'].apply(lambda x: clean_text(x))
df_test['sent2_clean'] = df_test['sentence_2'].apply(lambda x: clean_text(x))

In [165]:
vectorizer_test = CountVectorizer(vocabulary = tokens)
test1 = vectorizer_test.fit_transform(df_test['sent1_clean'])
test_sent1 = pd.DataFrame(test1.toarray(),
                            columns = vectorizer.get_feature_names())
print(test_sent1.shape)

test2 = vectorizer_test.fit_transform(df_test['sent2_clean'])
test_sent2 = pd.DataFrame(test2.toarray(),
                            columns = vectorizer.get_feature_names())
print(test_sent2.shape)


(4000, 11354)
(4000, 11354)


In [166]:
import numpy as geek

T1 = test_sent1.to_numpy()
T2 = test_sent2.to_numpy()

test_df_final = geek.add(T1, T2)
print(test_df_final.max())

#add data frames together to get overlapping words matrix

8


In [190]:
test_prediction = mlp.predict(test_df_final)
print(test_prediction)

newdf = pd.DataFrame(test_prediction)
newdf.to_csv('test_prediction.csv')

[0 0 0 ... 0 0 0]
